## Analyzing Energy Production with Elasticserach and Kibana Lens

In [79]:
import pandas as pd
import json
import time

In [120]:
data = pd.read_csv(filepath_or_buffer="all_energy_statistics.csv")
data['year'] = data.year.apply(lambda t: str(datetime.strptime(str(t), "%Y")))
#data['year'] = pd.to_datetime(data['year'])

In [121]:
## testing the format to insert into ES
data[0:10].to_json(path_or_buf="all_energy.json", orient="records")
converted = json.load(open("all_energy.json"))
helpers.bulk(es, convert_to_es_format(converted))

(10, [])

In [116]:
data[0:10]

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996-01-01,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995-01-01,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014-01-01,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013-01-01,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012-01-01,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates
5,Belgium,Additives and Oxygenates - Exports,2011-01-01,"Metric tons, thousand",25.0,NaN,additives_and_oxygenates
6,Belgium,Additives and Oxygenates - Exports,2010-01-01,"Metric tons, thousand",22.0,NaN,additives_and_oxygenates
7,Belgium,Additives and Oxygenates - Exports,2009-01-01,"Metric tons, thousand",45.0,NaN,additives_and_oxygenates
8,Czechia,Additives and Oxygenates - Exports,1998-01-01,"Metric tons, thousand",1.0,NaN,additives_and_oxygenates
9,Czechia,Additives and Oxygenates - Exports,1995-01-01,"Metric tons, thousand",7.0,NaN,additives_and_oxygenates


### Connnect to Elasticsearch

In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [43]:
# function to format data
def convert_to_es_format(docs):
    return [
        {
            "_index": "energy_stats",
            "_type": "_doc",
            "_source": doc
        }
        for doc in docs
    ]

In [124]:
total = len(data)
print(total)
cursor = 1
slider = 0

for i in range(total):
    if (cursor % 1000) == 0:
        print(slider, " ", cursor)
        # conver to json and index to es via bulk insert
        data[slider:cursor].to_json(path_or_buf="all_energy.json", orient="records")
        converted = json.load(open("all_energy.json"))
        helpers.bulk(es, convert_to_es_format(converted))
        slider = cursor
    
    if (total - i) < 1000:
        # insert last bulk
        data[slider:].to_json(path_or_buf="all_energy.json", orient="records")
        converted = json.load(open("all_energy.json"))
        helpers.bulk(es, convert_to_es_format(converted))
        break # ends the loop
        
    cursor+=1

1189482
0   1000
1000   2000
2000   3000
3000   4000
4000   5000
5000   6000
6000   7000
7000   8000
8000   9000
9000   10000
10000   11000
11000   12000
12000   13000
13000   14000
14000   15000
15000   16000
16000   17000
17000   18000
18000   19000
19000   20000
20000   21000
21000   22000
22000   23000
23000   24000
24000   25000
25000   26000
26000   27000
27000   28000
28000   29000
29000   30000
30000   31000
31000   32000
32000   33000
33000   34000
34000   35000
35000   36000
36000   37000
37000   38000
38000   39000
39000   40000
40000   41000
41000   42000
42000   43000
43000   44000
44000   45000
45000   46000
46000   47000
47000   48000
48000   49000
49000   50000
50000   51000
51000   52000
52000   53000
53000   54000
54000   55000
55000   56000
56000   57000
57000   58000
58000   59000
59000   60000
60000   61000
61000   62000
62000   63000
63000   64000
64000   65000
65000   66000
66000   67000
67000   68000
68000   69000
69000   70000
70000   71000
71000   72000
72000 

In [123]:
print(total)

1189482


## Next Steps:
* Import the file as csv (done)
* Convert the file to json (done)
- Install elasticsearch using docker (7.5.0); have access to Kibana Lens
- Create a mapping for the index before bulk indexing with the proper types
- Then index into elasticsearch in an index (bulk)
- Making sure we have chosen the proper types

## References

- [Elastic Stack 7.5 Released](https://www.elastic.co/blog/elastic-stack-7-5-0-released)
- [Wind, Solar and Nuclear Energy Production](https://www.kaggle.com/vtorreslopez/wind-solar-and-nuclear-energy-production)
- [Kibana Lens Overview: An easy, intuitive way to visualize Elasticsearch data](https://www.youtube.com/watch?v=6U7uoCyabcg)
- [International Energy Statistics](https://www.kaggle.com/unitednations/international-energy-statistics)